### In this file we are going to train our four models with different combinations of preprocessing methods and vectorization methods for each model, so we can find out which combination of preprocessing and vectorization methods suits better for every type of the models.

In [1]:
### imports 
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from config import path_to_data_folder

In [2]:
# make neccesary imports for preprocessing and vectorization
from functions_preprocessing import flow_preprocessing_1, flow_preprocessing_2, flow_preprocessing_3, flow_preprocessing_4, flow_preprocessing_5, flow_preprocessing_6, flow_preprocessing_7, flow_preprocessing_8, flow_preprocessing_9, flow_preprocessing_10, flow_preprocessing_11, flow_preprocessing_12, flow_preprocessing_13, flow_preprocessing_14, flow_preprocessing_15, flow_preprocessing_16, flow_preprocessing_17, flow_preprocessing_18, flow_preprocessing_19, flow_preprocessing_20
from functions_vectorization import TfidfVectorizer, CountVectorizer, Word2VecVectorizer, FastText

list_of_preprocessing_functions = [flow_preprocessing_1, flow_preprocessing_2, flow_preprocessing_3, flow_preprocessing_4, flow_preprocessing_5, flow_preprocessing_6, flow_preprocessing_7, flow_preprocessing_8, flow_preprocessing_9, flow_preprocessing_10, flow_preprocessing_11, flow_preprocessing_12, flow_preprocessing_13, flow_preprocessing_14, flow_preprocessing_15, flow_preprocessing_16, flow_preprocessing_17, flow_preprocessing_18, flow_preprocessing_19, flow_preprocessing_20]
list_of_vectorizers = [TfidfVectorizer, CountVectorizer, Word2VecVectorizer, FastText]

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/alexraudvee/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/alexraudvee/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/alexraudvee/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alexraudvee/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


ValueError:  cannot be opened for loading!

##### Change the path to the data folder according to your device

In [3]:
### PREPARE THE DATA FOR FITTING THE FUTURE MODELS
# Reading datasets

# path to the data folder (change for your device)
path_data = '/Users/alexraudvee/Desktop/lai-data/'

gender_df = pd.read_csv(f'{path_data}gender.csv')

jud_per_df = pd.read_csv(f'{path_data}judging_perceiving.csv')

political_df  = pd.read_csv(f'{path_data}political_leaning.csv')

# Split the dataset into training and testing sets
X = gender_df['post']  # Replace 'text_column' with the column containing text data
y = gender_df['female']  # Replace 'target_column' with the column containing target labels

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Create different pipelines for future model training and testing

In [4]:
# PIPELINES FOR LOGISTICREGRESSION
created_pipelines_LogisticRegression = {}

for vectorizer in list_of_vectorizers:
    for preprocessing_flow_function in list_of_preprocessing_functions:

        pipeline_name = f"pipeline_{preprocessing_flow_function.__name__}_{vectorizer.__name__}_Logreg"

        pipeline = Pipeline([
        ('preprocess', vectorizer(preprocessor=preprocessing_flow_function)),
        ('model', LogisticRegression())])

        created_pipelines_LogisticRegression[pipeline_name] = pipeline

# PIPELINES FOR MODEL2
created_pipelines_Model2 = {}

for vectorizer in list_of_vectorizers:
    for preprocessing_flow_function in list_of_preprocessing_functions:

        pipeline_name = f"pipeline_{preprocessing_flow_function.__name__}_{vectorizer.__name__}_Logreg"

        pipeline = Pipeline([
        ('preprocess', vectorizer(preprocessor=preprocessing_flow_function)),
        ('model', LogisticRegression())])

        created_pipelines_Model2[pipeline_name] = pipeline

# PIPELINES FOR MODEL3
created_pipelines_Model3 = {}

for vectorizer in list_of_vectorizers:
    for preprocessing_flow_function in list_of_preprocessing_functions:

        pipeline_name = f"pipeline_{preprocessing_flow_function.__name__}_{vectorizer.__name__}_Logreg"

        pipeline = Pipeline([
        ('preprocess', vectorizer(preprocessor=preprocessing_flow_function)),
        ('model', LogisticRegression())])

        created_pipelines_Model3[pipeline_name] = pipeline

# PIPELINES FOR MODEL4
created_pipelines_Model4 = {}

for vectorizer in list_of_vectorizers:
    for preprocessing_flow_function in list_of_preprocessing_functions:

        pipeline_name = f"pipeline_{preprocessing_flow_function.__name__}_{vectorizer.__name__}_Logreg"

        pipeline = Pipeline([
        ('preprocess', vectorizer(preprocessor=preprocessing_flow_function)),
        ('model', LogisticRegression())])

        created_pipelines_Model4[pipeline_name] = pipeline

In [5]:
pipeline_1 = created_pipelines_LogisticRegression['pipeline_flow_preprocessing_1_TfifdVectorizer_Logreg']
pipeline_1.fit(X_train, y_train)

TypeError: expected string or bytes-like object